#Lista 3
-------------


Objetivo: agrupar conjunto de observações de um banco de dados.

Banco utilizado: Wholesale customer

Descrição do dados:

1. FRESH: Gastos anuais (m.u.) em produtos frescos (Contínuo)
2. MILK: Gastos anuais (m.u.) em produtos lácteos (Contínuo)
3. GROCERY: Gastos anuais (m.u.) em produtos de mercearia (Contínuo)
4. FROZEN: Gastos anuais (m.u.) em produtos congelados (Contínuo)
5. DETERGENTS_PAPER: Gastos anuais (m.u.) em detergentes e produtos de papel (Contínuo)
6. DELICATESSEN: Gastos anuais (m.u.) em produtos de delicatessen (Contínuo)
7. CHANNEL: Canal dos clientes - Horeca (Hotel/Restaurante/Café) ou Varejo (Nominal)
8. REGION: Região dos clientes - Lisboa, Porto ou Outras (Nominal)

Exercício:

1. Carregamento e Análise Inicial:
* Carregue o banco de dados e analise suas features.
* Não utilize as variáveis Channel e Region.
* Realize a padronização dos dados.

**Observação**: Nos exercícios seguintes, utilize os dados padronizados.

2. Geração de Clusters:
* Gere 5 clusters utilizando os algoritmos K-means, Aglomerativo e DBSCAN (Utilize eps de 3000 e min_samples de 5).
3. Análise de Silhouette Score:
* Analise o resultado do Silhouette Score para cada agrupamento. Qual dentre os três é o melhor?

4. Decomposição de Componentes Principais (PCA):
* Realize a decomposição de componentes principais (PCA) com duas componentes.
* Qual o percentual da variabilidade é explicado com duas componentes?
* Construa um gráfico de pontos (scatterplot) onde no eixo x temos a primeira componente e no eixo y temos a segunda componente. Colora os pontos com o melhor cluster.

5. Método do Cotovelo (Elbow Method):
* Utilize o método do cotovelo junto com o algoritmo K-means para definir qual o melhor número de grupos.

## 1. Carga do banco
--------------------

In [1]:
import pandas as pd

df = pd.read_csv('../datasource/Wholesale customers data.csv')

In [2]:
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


In [4]:
df.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,1.322727,2.543182,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,0.468052,0.774272,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,1.000000,1.000000,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,1.000000,3.000000,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,2.000000,3.000000,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


In [5]:
df.isna().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

In [6]:
# Check correlation between features in df
df.corr()

# Plot  correlation matrix wit plotly
import plotly.express as px

fig = px.imshow(df.corr())
fig.show()


In [15]:
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Clusters
0,2,3,12669,9656,7561,214,2674,1338,1
1,2,3,7057,9810,9568,1762,3293,1776,1
2,2,3,6353,8808,7684,2405,3516,7844,1
3,1,3,13265,1196,4221,6404,507,1788,1
4,2,3,22615,5410,7198,3915,1777,5185,0
...,...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204,0
436,1,3,39228,1431,764,4510,93,2346,4
437,2,3,14531,15488,30243,437,14841,1867,2
438,1,3,10290,1981,2232,1038,168,2125,1


## 2. Geração dos clusters
--------------------
### 2.1. K-Means
--------------------

In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=45)

In [8]:
df['Clusters'] = kmeans.fit_predict(df)

centroids = kmeans.cluster_centers_
labels = df['Clusters']

In [9]:
print(f'Centroides {centroids}')
print(f'Labels { labels}')

Centroides [[1.20000000e+00 2.55238095e+00 2.17746000e+04 3.93420952e+03
  5.14920000e+03 4.18312381e+03 1.11254286e+03 1.69692381e+03]
 [1.19148936e+00 2.54042553e+00 6.01969787e+03 3.45541702e+03
  4.39926809e+03 2.39908085e+03 1.36218298e+03 1.00187660e+03]
 [1.93243243e+00 2.48648649e+00 5.27567568e+03 1.29892027e+04
  2.00090135e+04 1.63425676e+03 8.91101351e+03 1.90244595e+03]
 [2.00000000e+00 2.80000000e+00 2.56030000e+04 4.34606000e+04
  6.14722000e+04 2.63600000e+03 2.99742000e+04 2.70880000e+03]
 [1.09523810e+00 2.66666667e+00 5.05120952e+04 6.98752381e+03
  6.47809524e+03 1.02153810e+04 1.03052381e+03 4.90476190e+03]]
Labels 0      1
1      1
2      1
3      1
4      0
      ..
435    0
436    4
437    2
438    1
439    1
Name: Clusters, Length: 440, dtype: int32


In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

df_pca = pca.fit_transform(df.drop('Clusters', axis=1))

df_pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_pca_df['Clusters'] = df['Clusters']

In [11]:
import plotly.express as px

fig = px.scatter(df_pca_df, x='PC1', y='PC2', color='Clusters', title="Clusters gerados pelo K-Means",
                 labels={'PC1': 'Componente Principal 1', 'PC2': 'Componente Principal 2', 'Cluster': 'Cluster'},
                 color_continuous_scale='Viridis')

# Mostrar o gráfico
fig.show()


In [12]:
print(f'Variancia explicada pelos componentes {pca.explained_variance_ratio_}')

Variancia explicada pelos componentes [0.45961362 0.40517227]


In [13]:
from sklearn.metrics import silhouette_score, silhouette_samples

# Calcular o índice de Silhouette para todos os pontos
silhouette_avg = silhouette_score(df_pca_df, labels)

print(f"Índice médio de Silhouette: {silhouette_avg}")

sample_silhouette_values = silhouette_samples(df_pca_df, labels)

print(f"Valores de Silhouette para cada Ponto Individual: {sample_silhouette_values}")

Índice médio de Silhouette: 0.45305021988292643
Valores de Silhouette para cada Ponto Individual: [ 0.16095296  0.29738874  0.4338464   0.06500155  0.57303152  0.48395951
  0.32523164  0.51363474  0.66292358  0.5024772   0.11112791  0.21573645
  0.37819901  0.34694389  0.4437442   0.5246245  -0.01849781  0.67152596
  0.37289256  0.57643872  0.39004161  0.65881421  0.44704307  0.13662403
  0.38647133  0.1828753   0.50495641  0.06528494  0.55853482  0.28800008
  0.38624149  0.69687656  0.5644788   0.55192149  0.66839785  0.2925044
  0.56850406 -0.05645454  0.52601348  0.47836434  0.61460582  0.49121429
  0.1209761   0.55285358  0.3317899   0.57338822  0.57439146  0.56118129
  0.2583701   0.52108417  0.64849146  0.64894614  0.14114843  0.07857238
  0.5749432   0.68133304  0.48187503 -0.00998132  0.12492972  0.57699113
  0.55939931  0.5988564   0.53478533  0.35822257  0.68949253  0.20801669
  0.56843979  0.32971279  0.68049151  0.60275799  0.43988931  0.15751929
  0.68579712  0.51550727  0

### 2.2. Aglomerativo
--------------------

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

df_witou_cluster_column = df.drop(columns=["Clusters"])

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_witou_cluster_column)

agg_cluster = AgglomerativeClustering(n_clusters=5, linkage='ward')

df['Predicted_Clusters'] = agg_cluster.fit_predict(X_scaled)

In [17]:
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Clusters,Predicted_Clusters
0,2,3,12669,9656,7561,214,2674,1338,1,2
1,2,3,7057,9810,9568,1762,3293,1776,1,2
2,2,3,6353,8808,7684,2405,3516,7844,1,2
3,1,3,13265,1196,4221,6404,507,1788,1,0
4,2,3,22615,5410,7198,3915,1777,5185,0,2
...,...,...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204,0,0
436,1,3,39228,1431,764,4510,93,2346,4,0
437,2,3,14531,15488,30243,437,14841,1867,2,2
438,1,3,10290,1981,2232,1038,168,2125,1,0


In [18]:
import plotly.express as px

# Criando o gráfico de dispersão com Plotly
fig = px.scatter(
    df,
    x="Fresh",
    y="Milk",
    color="Predicted_Clusters",
    title="Clusters Preditos com Agglomerative Clustering",
    labels={"Fresh": "Fresh", "Milk": "Milk", "Predicted_Clusters": "Clusters"},
    color_continuous_scale="Viridis",
    size_max=15
)

fig.update_layout(
    xaxis_title="Fresh",
    yaxis_title="Milk",
    legend_title="Clusters",
    template="plotly_white"
)

fig.show()


##2.3. DBSCAN
--------------------

In [ ]:
from sklearn.cluster import DBSCAN

# Aplicando o DBSCAN
dbscan = DBSCAN(eps=1.5, min_samples=2)  # Ajustando parâmetros para este conjunto de dados

df['DBSCAN_Clusters'] = dbscan.fit_predict(X_scaled)

# Criando gráfico para exibir os resultados
fig = px.scatter(
    df,
    x="Fresh",
    y="Milk",
    color="DBSCAN_Clusters",
    title="Clusters Preditos com DBSCAN",
    labels={"Fresh": "Fresh", "Milk": "Milk", "DBSCAN_Clusters": "Clusters"},
    color_continuous_scale="Plasma",
)

fig.update_layout(
    xaxis_title="Fresh",
    yaxis_title="Milk",
    legend_title="Clusters",
    template="plotly_white"
)

fig.show()


## 3 Análise sillhouete score
---------------------------

In [ ]:
from sklearn.metrics import silhouette_score

agg_silhouette = silhouette_score(X_scaled, df['Predicted_Clusters'])

dbscan_clusters = df['DBSCAN_Clusters']

valid_dbscan_mask = dbscan_clusters != -1

dbscan_silhouette = silhouette_score(X_scaled[valid_dbscan_mask], dbscan_clusters[valid_dbscan_mask])

agg_silhouette, dbscan_silhouette


(np.float64(0.3178066486005939), np.float64(0.33573149594028695))

### Resultado

O DBSCAN apresenta um Silhouette Score levemente melhor (0.336 contra 0.318), o que indica que ele pode ser uma escolha melhor para o agrupamento deste conjunto de dados.

## 4. Visualização
---------------------------
## 4.1. PCA
---------------------------

In [21]:
from sklearn.decomposition import PCA

# Aplicando PCA para reduzir para 2 dimensões
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Adicionando os componentes principais ao DataFrame
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]

# Criando um gráfico com os clusters Aglomerativos
fig_agg = px.scatter(
    df,
    x="PCA1",
    y="PCA2",
    color="Predicted_Clusters",
    title="Clusters Aglomerativos após PCA",
    labels={"PCA1": "Componente Principal 1", "PCA2": "Componente Principal 2", "Predicted_Clusters": "Clusters"},
    color_continuous_scale="Viridis"
)

# Criando um gráfico com os clusters do DBSCAN
fig_dbscan = px.scatter(
    df,
    x="PCA1",
    y="PCA2",
    color="DBSCAN_Clusters",
    title="Clusters DBSCAN após PCA",
    labels={"PCA1": "Componente Principal 1", "PCA2": "Componente Principal 2", "DBSCAN_Clusters": "Clusters"},
    color_continuous_scale="Plasma"
)

fig_agg.show()
fig_dbscan.show()


## 5. Método elbow
---------------------------

In [23]:
from sklearn.cluster import KMeans
import plotly.graph_objects as go

# Calculando a inércia para diferentes números de clusters (1 a 10)
inertia_values = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia_values.append(kmeans.inertia_)
    
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(k_values),
    y=inertia_values,
    mode='lines+markers',
    marker=dict(color='blue', size=8),
    line=dict(dash='dash', color='blue'),
    name='Inércia (WCSS)'
))

fig.update_layout(
    title='Método Elbow para Determinar o Número Ideal de Clusters',
    xaxis_title='Número de Clusters (k)',
    yaxis_title='Inércia (WCSS)',
    template='plotly_white',
    xaxis=dict(tickmode='linear', tick0=1, dtick=1),
    yaxis=dict(showgrid=True)
)

fig.show()